In [7]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

import xgboost as xgb

In [2]:
challenger_games = pd.read_csv("final_data/Challenger_Ranked_Games.csv")
grandmaster_games = pd.read_csv("final_data/GrandMaster_Ranked_Games.csv")
master_games = pd.read_csv("final_data/Master_Ranked_Games.csv")

In [39]:
def per_minute(raw_df):
    properties = ["WardPlaced", "Wardkills", "Kills", "ChampionDamageDealt", "TotalGold", "TotalMinionKills", "JungleMinionKills", "TotalHeal", "ObjectDamageDealt"]
    raw_df["duration"] = raw_df["duration"] / 60
    game_duration = raw_df["duration"]
    for column in properties:
        raw_df[column] = raw_df[column] / game_duration
    return raw_df

In [40]:
def difference(blue_df, red_df):
    properties = ["WardPlaced", "Wardkills", "Kills", "ChampionDamageDealt", "TotalGold", "TotalMinionKills", "JungleMinionKills", "TotalHeal", "ObjectDamageDealt"]
    for column in properties:
        blue_columns_values = []
        red_columns_values = []
        for index, row in blue_df.iterrows():
            blue_columns_values.append(blue_df.loc[index][column] - red_df.loc[index][column])
            red_columns_values.append(red_df.loc[index][column] - blue_df.loc[index][column])
        blue_df[column] = blue_columns_values
        red_df[column] = red_columns_values
        blue_columns_values = []
        red_columns_values = []        
    return([blue_df, red_df])

In [41]:
def preprocessing(raw_df):
    column_list = []
    blue_df = raw_df.loc[:, raw_df.columns.str.startswith('blue')]
    for column in blue_df.columns:
        column_list.append(column[4:])
    
    blue_df.columns = column_list
    blue_df["duration"] = raw_df.get("gameDuraton")
    blue_df["team"] = "b"
    blue_df = per_minute(blue_df)
    
    column_list = []
    red_df = raw_df.loc[:, raw_df.columns.str.startswith('red')]
    for column in red_df.columns:
        column_list.append(column[3:])
    
    red_df.columns = column_list
    red_df["duration"]  = raw_df.get("gameDuraton")
    red_df["team"] = "r"
    red_df = per_minute(red_df)
    
    [blue_df, red_df] = difference(blue_df, red_df)
    
    complete_df = blue_df.append(red_df)
    complete_df = complete_df.drop(columns = ["TotalLevel"])
    complete_df["AvgLevel"] = complete_df["AvgLevel"] / 18
    complete_df["TowerKills"] = complete_df["TowerKills"] / 11
    complete_df = complete_df.reset_index()
    return complete_df

In [90]:
output = preprocessing(challenger_games[:5000])
output

c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,index,Wins,FirstBlood,FirstTower,FirstBaron,FirstDragon,FirstInhibitor,DragonKills,BaronKills,TowerKills,...,ChampionDamageDealt,TotalGold,TotalMinionKills,AvgLevel,JungleMinionKills,KillingSpree,TotalHeal,ObjectDamageDealt,duration,team
0,0,0,1,0,0,0,0,0,0,0.000000,...,-769.931973,-581.904762,-4.217687,0.622222,-1.496599,4,-348.117914,-2484.716553,22.050000,b
1,1,1,0,0,0,1,0,2,0,0.363636,...,730.068337,179.316629,-1.822323,0.700000,-1.047836,3,71.435080,1421.321185,21.950000,b
2,2,0,0,0,0,0,0,0,0,0.000000,...,-436.673820,-800.536481,-6.051502,0.466667,-1.609442,0,-387.682403,-2546.137339,15.533333,b
3,3,0,0,0,1,1,0,1,1,0.181818,...,-693.775024,-202.507150,-1.658723,0.888889,0.600572,3,-1160.934223,-1635.357483,34.966667,b
4,4,1,1,1,1,1,1,2,2,1.000000,...,84.215017,9.189420,0.947099,0.922222,0.639932,5,-1261.177474,797.252560,39.066667,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4995,0,0,0,0,1,0,3,0,0.181818,...,-529.520458,-266.924769,-1.240651,0.888889,-3.510779,10,-0.686318,-1102.014958,37.883333,r
9996,4996,1,1,1,0,1,0,1,0,0.181818,...,953.828208,631.431601,4.772004,0.588889,4.326617,6,60.063627,911.898197,15.716667,r
9997,4997,0,1,0,0,1,0,1,0,0.181818,...,-298.518519,-212.633745,-1.604938,0.700000,-0.123457,6,-135.925926,-807.654321,24.300000,r
9998,4998,0,0,1,0,0,0,3,0,0.363636,...,-48.107256,-18.359621,-2.176656,0.822222,1.419558,9,-522.113565,212.145110,31.700000,r


In [91]:
output.dtypes

index                    int64
Wins                     int64
FirstBlood               int64
FirstTower               int64
FirstBaron               int64
FirstDragon              int64
FirstInhibitor           int64
DragonKills              int64
BaronKills               int64
TowerKills             float64
InhibitorKills           int64
WardPlaced             float64
Wardkills              float64
Kills                  float64
Death                    int64
Assist                   int64
ChampionDamageDealt    float64
TotalGold              float64
TotalMinionKills       float64
AvgLevel               float64
JungleMinionKills      float64
KillingSpree             int64
TotalHeal              float64
ObjectDamageDealt      float64
duration               float64
team                    object
dtype: object

In [92]:
X = output.drop(["Wins"], axis = 1)
y = output.Wins

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [94]:
output.columns

Index(['index', 'Wins', 'FirstBlood', 'FirstTower', 'FirstBaron',
       'FirstDragon', 'FirstInhibitor', 'DragonKills', 'BaronKills',
       'TowerKills', 'InhibitorKills', 'WardPlaced', 'Wardkills', 'Kills',
       'Death', 'Assist', 'ChampionDamageDealt', 'TotalGold',
       'TotalMinionKills', 'AvgLevel', 'JungleMinionKills', 'KillingSpree',
       'TotalHeal', 'ObjectDamageDealt', 'duration', 'team'],
      dtype='object')

In [95]:
cat_feat = [
    "FirstBlood",
    "FirstTower",
    "FirstBaron",
    "FirstDragon",
    "FirstInhibitor"
]
cat_transformer = Pipeline(steps = [
    ("onehot", OneHotEncoder())
])


log_feat = ['DragonKills', 'BaronKills',
       'TowerKills', 'InhibitorKills', 'WardPlaced', 'Wardkills', 'Kills',
       'Death', 'Assist', 'ChampionDamageDealt', 'TotalGold',
       'TotalMinionKills', 'AvgLevel', 'JungleMinionKills', 'KillingSpree',
       'TotalHeal', 'ObjectDamageDealt', 'duration']
log_tranformer = Pipeline(steps = [
    ("log", FunctionTransformer(lambda x: x))
])



preproc = ColumnTransformer(transformers = [("cat", cat_transformer, cat_feat), 
                                            ("log", log_tranformer, log_feat)])

pl = Pipeline(steps = [("preprocessor", preproc), ("xgbclassifier", xgb.XGBClassifier())])

pl.fit(X_train, y_train)

preds = pl.predict(X_test)

In [96]:
preds

array([1, 0, 1, ..., 1, 0, 1], dtype=int64)

In [97]:
y_test

6252    1
4684    0
1731    1
4742    0
4521    1
       ..
6412    1
8285    0
7853    1
1095    0
6929    1
Name: Wins, Length: 2000, dtype: int64

In [98]:
accuracy_score(preds, y_test)

0.986